In [194]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [195]:
x = torch.load("../../../data/data.pt")
y = torch.load("../../../data/ages.pt")
n = x.shape[0]
y = torch.reshape(y,(n,1))
print(x.shape,y.shape)

torch.Size([536, 3, 224, 224]) torch.Size([536, 1])


In [196]:
fcp_x = torch.load("../../../data/fcp_data.pt")
fcp_y = torch.load("../../../data/fcp_ages.pt")
n = fcp_x.shape[0]
fcp_y = torch.reshape(fcp_y,(n,1))
print(fcp_x.shape,fcp_y.shape)

torch.Size([1034, 3, 224, 224]) torch.Size([1034, 1])


In [197]:
#Normalizing the data (according to the stats provided) before feeding into the pretrained model
x_normalized = torch.Tensor(x.shape)
fcp_x_normalized = torch.Tensor(fcp_x.shape)
for i in range(n):
#     x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]).__call__(x[i,:,:])
    fcp_x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]).__call__(fcp_x[i,:,:])
    

In [198]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [199]:
dataset = utils.TensorDataset(fcp_x_normalized, fcp_y)

In [200]:
# dataloaders = {i: torch.utils.data.DataLoader(dataset, batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for i in ['train', 'val']}
train_indices = [i for i in range(n) if i % 2 == 0]
val_indices = [i for i in range(n) if i % 2 == 1]
test_indices = [i for i in range(n) if i % 3 == 2]
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(train_indices),num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(val_indices),num_workers=4)

In [201]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device,dtype = torch.float)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))

            epoch_loss = running_loss / n
            epoch_offset = running_offset / n

            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [202]:
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [204]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 84.5503 Diff: 4.4445
val Loss: 105.9489 Diff: 6.0469

Epoch 1/9
----------
train Loss: 83.4657 Diff: 4.4412
val Loss: 208.4749 Diff: 9.6933

Epoch 2/9
----------
train Loss: 82.1749 Diff: 4.5311
val Loss: 214.2148 Diff: 9.7800

Epoch 3/9
----------
train Loss: 79.1791 Diff: 4.3484
val Loss: 108.4029 Diff: 6.2419

Epoch 4/9
----------
train Loss: 74.5254 Diff: 4.2587
val Loss: 631.7717 Diff: 16.7052

Epoch 5/9
----------
train Loss: 73.5969 Diff: 4.0609
val Loss: 177.3184 Diff: 8.8559

Epoch 6/9
----------
train Loss: 65.1400 Diff: 3.8821
val Loss: 342.0487 Diff: 12.3176

Epoch 7/9
----------
train Loss: 68.9629 Diff: 4.0992
val Loss: 116.9383 Diff: 4.3838

Epoch 8/9
----------
train Loss: 71.2870 Diff: 4.0400
val Loss: 117.9853 Diff: 4.3606

Epoch 9/9
----------
train Loss: 67.3094 Diff: 3.9845
val Loss: 131.6733 Diff: 4.8346

Training complete in 2m 58s
Best val Diff: 4.360609


In [160]:
# torch.save(model_ft.state_dict(), "best_regression_model.pth")

In [182]:
print(model_ft(fcp_x[test_indices,:].to(device)))

tensor([[23.8213],
        [23.8092],
        [23.7417],
        [23.7096],
        [23.6990],
        [23.7326],
        [24.0736],
        [33.5329],
        [32.9792],
        [32.3286],
        [31.9377],
        [32.3584],
        [32.2043],
        [33.0160],
        [23.3244],
        [23.3342],
        [23.3596],
        [23.2126],
        [23.3149],
        [23.3081],
        [23.3311],
        [23.3022],
        [23.3694],
        [23.3060],
        [23.3412],
        [23.2774],
        [23.3428],
        [23.3812],
        [23.3305],
        [23.4058],
        [23.3518],
        [23.3784],
        [23.3487],
        [23.2914],
        [23.3836],
        [23.3232],
        [23.3734],
        [23.3705],
        [23.4053],
        [23.3669],
        [23.3986],
        [23.3513],
        [23.3641],
        [23.3360],
        [23.3577],
        [23.3598],
        [23.4642],
        [23.2518],
        [23.2235],
        [23.3034],
        [23.3616],
        [23.3694],
        [23.

In [179]:
def check_error(data,labels,model):
    print('Checking error on test set')   
    total_abs_err = 0.0
    num_samples = labels.shape[0]
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device,dtype=dtype)
        scores = model(data)
        total_abs_err += (torch.abs(labels - scores).sum())
        mean_abs_err = float(total_abs_err) / num_samples
        print('Got error %.2f' % (mean_abs_err))

In [181]:
check_error(fcp_x[test_indices,:], fcp_y[test_indices],model_ft)

Checking error on test set
tensor([[23.8092],
        [23.7417],
        [23.7096],
        [23.6990],
        [23.7326],
        [24.0736],
        [33.5329],
        [32.9792],
        [32.3286]], device='cuda:0') tensor([[26.],
        [40.],
        [24.],
        [26.],
        [23.],
        [33.],
        [29.],
        [36.],
        [22.]], device='cuda:0')
Got error 8.18


In [188]:
# torch.save(model_ft,"the_whole_model.pth")